# Signal Analysis with Snowtrail

This notebook demonstrates how to analyze and visualize Snowtrail signals for trading research.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from snowtrail import Snowtrail

client = Snowtrail()

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Load Historical Signal Data

In [ ]:
# Fetch full history of GBSI-US system stress
df = client.gbsi_us.system_stress(latest=False, limit=500)

# Convert date column to datetime
df['week_ending'] = pd.to_datetime(df['week_ending'])
df = df.sort_values('week_ending').reset_index(drop=True)

print(f"Data range: {df['week_ending'].min()} to {df['week_ending'].max()}")
print(f"Total observations: {len(df)}")
df.head()

## Regime Distribution

In [ ]:
# Count regime occurrences
regime_counts = df['stress_regime_label'].value_counts()

fig, ax = plt.subplots()
regime_counts.plot(kind='bar', ax=ax, color=['green', 'blue', 'orange', 'red'][:len(regime_counts)])
ax.set_title('Stress Regime Distribution')
ax.set_xlabel('Regime')
ax.set_ylabel('Count (weeks)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Signal Time Series

In [ ]:
# Plot stress regime over time
fig, ax = plt.subplots()

ax.plot(df['week_ending'], df['stress_regime'], marker='o', markersize=3, linewidth=1)
ax.set_title('GBSI-US System Stress Over Time')
ax.set_xlabel('Date')
ax.set_ylabel('Stress Regime (0-4)')
ax.set_ylim(-0.5, 4.5)

# Add regime labels
ax.axhline(y=0, color='green', linestyle='--', alpha=0.3, label='Loose')
ax.axhline(y=2, color='blue', linestyle='--', alpha=0.3, label='Balanced')
ax.axhline(y=4, color='red', linestyle='--', alpha=0.3, label='Tight')

plt.tight_layout()
plt.show()

## Regime Transitions

In [ ]:
# Calculate regime transitions
df['regime_change'] = df['stress_regime'].diff().fillna(0)
df['regime_up'] = df['regime_change'] > 0
df['regime_down'] = df['regime_change'] < 0

print(f"Weeks with regime increase: {df['regime_up'].sum()}")
print(f"Weeks with regime decrease: {df['regime_down'].sum()}")
print(f"Weeks unchanged: {(df['regime_change'] == 0).sum()}")

## Combine with Features

In [ ]:
# Load storage inventory feature
storage_df = client.gbsi_us.storage_inventory(limit=500)
storage_df['week_ending'] = pd.to_datetime(storage_df['week_ending'])

# Merge with signal data
merged = df.merge(storage_df[['week_ending', 'inventory_bcf', 'deviation_from_normal_pct']], 
                  on='week_ending', how='left')

merged.head()

In [ ]:
# Scatter plot: storage deviation vs stress regime
fig, ax = plt.subplots()

scatter = ax.scatter(merged['deviation_from_normal_pct'], merged['stress_regime'], 
                     c=merged['stress_regime'], cmap='RdYlGn_r', alpha=0.6)
ax.set_xlabel('Storage Deviation from Normal (%)')
ax.set_ylabel('Stress Regime')
ax.set_title('Storage Deviation vs Stress Regime')
plt.colorbar(scatter, label='Stress Regime')
plt.tight_layout()
plt.show()

## Event Analysis

In [ ]:
# Load storage surprise events
events_df = client.gbsi_us.storage_surprise(limit=100)
events_df['report_date'] = pd.to_datetime(events_df['report_date'])

print(f"Total storage surprise events: {len(events_df)}")
events_df.head()

In [ ]:
# Distribution of surprise direction
if 'surprise_direction' in events_df.columns:
    surprise_counts = events_df['surprise_direction'].value_counts()
    print("Surprise Direction Distribution:")
    print(surprise_counts)

## Export for Further Analysis

In [ ]:
# Export merged dataset to CSV
# merged.to_csv('gbsi_us_analysis.csv', index=False)
# print("Exported to gbsi_us_analysis.csv")

## Next Steps

- Merge with price data for backtesting
- Build trading rules based on regime transitions
- Analyze regime persistence and mean reversion